In [1]:
import pandas as pd
import numpy as np
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
data_path = './data/'
!ls -lthr data/

total 2.8G
-rw-rw-r-- 1 sam sam 507M Mar 14 23:57 test.csv
-rw-rw-r-- 1 sam sam  51M Mar 14 23:57 submission_popular.csv
-rw-rw-r-- 1 sam sam 246M Mar 14 23:57 item_metadata.csv
-rw-rw-r-- 1 sam sam 2.0G Mar 14 23:57 train.csv
-rw-rw-r-- 1 sam sam 1.2M Mar 15 22:32 result.csv


In [3]:
%%time
nrows = 10000
# nrows = None
train = pd.read_csv(data_path+'train.csv', nrows=nrows)

CPU times: user 25.3 ms, sys: 4.55 ms, total: 29.9 ms
Wall time: 28.9 ms


In [4]:
test = pd.read_csv(data_path+'test.csv', nrows=1000)
test.shape

(1000, 12)

In [5]:
# test_click_session_ids = test[test.impressions.notnull()].session_id.unique()

In [6]:
# ts = np.random.choice(test_click_session_ids, 1)
# test[test.session_id==ts[0]]

In [7]:
# train_click_ids = train[train.impressions.notnull()].session_id.unique()

In [8]:
# ts = np.random.choice(train_click_ids, 1)
# train[train.session_id==ts[0]]

In [9]:
# train.groupby('session_id').apply(lambda x: x.iloc[-1]['reference']).isna().sum()

In [10]:
# train.groupby('session_id').apply(lambda x: x.iloc[-1]['impressions']).isna().sum()

### clip sessions off to last click

In [11]:
%%time
# get all rows upto the last clickout action (some rows has reference but it's not clickout action)
def up_to_last_click(grp):
    check = grp.action_type == 'clickout item'
    if check.sum() != 0:
        return grp.iloc[:np.argwhere(check)[-1][0]+1]
    else:
        return grp
        
train = train.groupby('session_id').apply(up_to_last_click).reset_index(drop=True)

/home/sam/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


CPU times: user 315 ms, sys: 7.54 ms, total: 322 ms
Wall time: 327 ms


In [12]:
%%time
# get utc timestamp
train['ts'] = train['timestamp'].apply(lambda t: datetime.datetime.utcfromtimestamp(t))

CPU times: user 3.34 ms, sys: 0 ns, total: 3.34 ms
Wall time: 3.31 ms


In [13]:
%%time
# only look at sessions with clickouts (for now)
# first filter out sessions that does not have a click-out
def check_clickout(grp):
     # sessions has clickouts
    has_clickout = 'clickout item' in grp['action_type'].unique()
    # last row has reference and it's not nan
    has_ref = ((grp['action_type'].iloc[-1] == 'clickout item') & 
               (grp.iloc[-1][['impressions', 'reference', 'prices']].isna().sum()==0))
#                (pd.notnull(grp.iloc[-1][])) &
#                (pd.notnull(grp.iloc[-1]['prices'])))
    return has_clickout & has_ref
    
clicked = train.groupby('session_id').apply(check_clickout)
click_session_ids = clicked[clicked].index
# filter
train = train[train.session_id.isin(click_session_ids)].reset_index(drop=True)

CPU times: user 488 ms, sys: 179 µs, total: 488 ms
Wall time: 487 ms


In [ ]:
train.

### meta info

In [14]:
%%time
meta = pd.read_csv(data_path+'item_metadata.csv', nrows=nrows)
meta['properties'] = meta['properties'].str.lower()

CPU times: user 32.6 ms, sys: 149 µs, total: 32.7 ms
Wall time: 32.2 ms


In [15]:
%%time
# checkout properties of items
def get_property_count(row):
    return len(row.split('|'))

item_p_ctn = meta['properties'].apply(get_property_count)
item_p_ctn.describe()

CPU times: user 10.2 ms, sys: 3.55 ms, total: 13.7 ms
Wall time: 13.3 ms


In [16]:
%%time
def get_property(row):
    return row.lower().split('|')
meta['ps'] = meta['properties'].apply(get_property)
# numer of properties
meta['nprop'] = meta.ps.str.len()
# star ratings
meta['star'] = meta.properties.str.extract('[\|](\d) star')
meta['star'] = meta['star'].astype(float)

CPU times: user 31.2 ms, sys: 3.82 ms, total: 35.1 ms
Wall time: 34.3 ms


In [17]:
%%time
# create rating columns
ratings = ['good rating', 'satisfactory rating', 'excellent rating']
for r in ratings:
    meta[r.replace(' ', '_')] = meta.properties.str.findall(f'\|{r}').str.len()

CPU times: user 24.1 ms, sys: 0 ns, total: 24.1 ms
Wall time: 23.7 ms


In [18]:
# total_ps = []
# for p in meta['ps'].values:
#     total_ps += p

In [19]:
# total_ps

In [20]:
# pd.value_counts(total_ps).iloc[:20].plot(kind='bar')

In [21]:
meta.head()

,item_id,properties,ps,nprop,star,good_rating,satisfactory_rating,excellent_rating
0,5121,terrace (hotel)|lift|childcare|from 3 stars|ra...,"[terrace (hotel), lift, childcare, from 3 star...",54,3.0,1,1,0
1,5555,terrace (hotel)|lift|from 3 stars|radio|from 4...,"[terrace (hotel), lift, from 3 stars, radio, f...",77,5.0,1,1,1
2,5684,terrace (hotel)|lift|from 3 stars|radio|4 star...,"[terrace (hotel), lift, from 3 stars, radio, 4...",62,4.0,1,1,0
3,5777,lift|childcare|from 3 stars|radio|4 star|from ...,"[lift, childcare, from 3 stars, radio, 4 star,...",69,4.0,1,1,0
4,6044,terrace (hotel)|childcare|from 3 stars|radio|b...,"[terrace (hotel), childcare, from 3 stars, rad...",45,3.0,1,1,0


In [55]:
meta.index[0]

5121

In [22]:
# meta.shape

In [23]:
meta = (meta[['item_id', 'nprop', 'star', 'good_rating', 'satisfactory_rating', 'excellent_rating']]
        .set_index('item_id'))

### Create session features
---

In [24]:
%%time
# find out if same customers have multiple sessions
s = train.drop_duplicates(subset=['user_id', 'session_id'])
s.session_id.duplicated().sum()
# seems like not

CPU times: user 4.09 ms, sys: 22 µs, total: 4.11 ms
Wall time: 3.86 ms


In [25]:
# train.current_filters[:1000].unique()

In [26]:
%%time
def get_filters(x):
    if type(x) == str or type(x) == list:
        return x.split('|')
    else:
        return np.nan
def get_impressions(x):
    if type(x) == str:
        return x.split('|')
    else:
        return np.nan
    
    
train['filters'] = train.current_filters.str.split('|')
train['nfilters'] = train.filters.str.len()
train['imps_list'] = train.impressions.str.split('|')
nn_mask = train['imps_list'].notnull()
train.loc[nn_mask, 'imps_list'] = train.loc[nn_mask, 'imps_list'].apply(lambda x: [int(i) for i in x])
train['nimps'] = train.imps_list.str.len()

CPU times: user 12.3 ms, sys: 83 µs, total: 12.4 ms
Wall time: 11.4 ms


In [58]:
train.imps_list.iloc[1][0]

94860

In [27]:
train.nimps.describe()

count    810.000000
mean      22.914815
std        5.083618
min        1.000000
25%       25.000000
50%       25.000000
75%       25.000000
max       25.000000
Name: nimps, dtype: float64

In [28]:
# no switch of devices during session
(train.groupby('session_id')['device'].nunique()!=1).sum()

0

In [29]:
# differnet city during session
(train.groupby('session_id')['city'].nunique()!=1).sum()

44

In [30]:
%%time
# some custom funcs used in agggregation
def mean_dwell_time(x):
    if len(x) == 1:
        return 0
    else:
        return np.mean(np.diff(np.sort(x)))
    
def var_dwell_time(x):
    if len(x) == 1:
        return 0
    else:
        return np.var(np.diff(np.sort(x)))
    
def get_first(x):
    return x.iloc[0]

def get_last(x):
    return x.iloc[-1]

def n_clickouts(x):
    return (x=='clickout item').sum()

def click_rel_pos_avg(x):
    return np.mean(np.argwhere((x=='clickout item')))/len(x)

# define some aggs
session_aggs = {'timestamp': [np.ptp, mean_dwell_time, var_dwell_time],
                'step': ['max'],
                'action_type': ['nunique', n_clickouts, click_rel_pos_avg],
                'city': ['nunique', get_first],
                'platform': [get_first],
                'device': [get_first],
                'nfilters': ['mean', 'max', 'min', get_last],
                'nimps': ['max']
               }

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 6.44 µs


In [31]:
session_grp = train.groupby('session_id')

In [32]:
%%time
session_fts = session_grp.agg(session_aggs)
session_fts.columns = ['_'.join(col).strip() for col in session_fts.columns.values]

/home/sam/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/sam/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


CPU times: user 325 ms, sys: 4.19 ms, total: 329 ms
Wall time: 307 ms


In [33]:
# session_fts.columns.values
session_fts.head()

,timestamp_ptp,timestamp_mean_dwell_time,timestamp_var_dwell_time,step_max,action_type_nunique,action_type_n_clickouts,action_type_click_rel_pos_avg,city_nunique,city_get_first,platform_get_first,device_get_first,nfilters_mean,nfilters_max,nfilters_min,nfilters_get_last,nimps_max
session_id,,,,,,,,,,,,,,,,
002eb79ce41a3,58,58.000000,0.000000,2,2,1,0.500000,1,"Batu Ferringhi, Malaysia",MY,mobile,NaN,NaN,NaN,NaN,25.0
018d2fb48015e,94,23.500000,1236.750000,5,4,1,0.800000,1,"Tokyo, Japan",JP,mobile,1.0,1.0,1.0,NaN,25.0
028c5e1046a35,19,19.000000,0.000000,2,1,2,0.250000,1,"Ito, Japan",JP,mobile,NaN,NaN,NaN,NaN,25.0
03db51d40d211,305,23.461538,3120.710059,14,5,1,0.928571,1,"Hermosillo, Mexico",MX,mobile,NaN,NaN,NaN,NaN,18.0
044b409e8d102,18,9.000000,81.000000,3,2,1,0.666667,1,"Coolangatta, Australia",AU,tablet,NaN,NaN,NaN,NaN,25.0


In [34]:
# train[train.session_id=='62991f7c78f27']

### Create clickout features
---

In [35]:
import time

class Timer:    
    def __init__(self, task, profile):
        self.profile = profile
        self.task = task
        if self.task not in self.profile.keys():
            self.profile[self.task] = 0
        
    def __enter__(self):
        self.start = time.process_time()
        return self

    def __exit__(self, *args):
        self.end = time.process_time()
        self.interval = self.end - self.start
        self.profile[self.task] += np.round(self.interval, decimals=5)

In [36]:
session_grp = train.groupby('session_id')

In [37]:
# train[train.session_id=='62991f7c78f27']

In [38]:
def get_profile(profile):
    profile_df = pd.Series(profile).rename_axis(index='task').reset_index(name='sec')
    profile_df['per'] = (profile_df['sec']/profile_df['sec'].sum()*100).round(2)
    return profile_df

In [52]:
%%time
# 1) has the items appeared in previous impressions, and clickout
# session_grp
profile = {}
def compute_clicks(grp):
    with Timer('select_rows', profile) as t:
        last_row = grp.iloc[-1]
        above = grp.iloc[:-1]
    with Timer('exclude_nans', profile) as t:
        # get previous appeard impressions
        prev = above[above['impressions'].notnull()]
    with Timer('get_imp_list', profile) as t:
        prev_imps = prev['imps_list']
        unique_imps = [j for i in prev_imps for j in i]
    
    with Timer('get_price', profile) as t:
        imp_l = last_row['imps_list']
        prices = last_row['prices'].split('|')
        prices = [int(p) for p in prices]
        # whether the impression appeared before
        appeared = [int(i in unique_imps) for i in imp_l]
        # the location of the impression
        locs = list(range(len(imp_l)))
    
    with Timer('create_df', profile) as t:
    # build the df
        result = pd.DataFrame({'appeared': appeared, 'location': locs, 'price': prices}, index=imp_l)
        result.index.name = 'item_id'
    with Timer('rel_price_rank', profile) as t:
        price_ind = np.argsort(result['price'].values) + 1
        result['rel_price_rank'] = price_ind/len(imp_l)
#         result['rel_price_rank'] = result[['location', 'price']].sort_values(by='price')['location']/len(imp_l)
    
    with Timer('compute_mean_median', profile) as t:
        result['price_mean'] = np.mean(result['price'])
        result['price_median'] = np.median(result['price'])

    with Timer('compute_diff', profile) as t: 
        result_price = result['price'].values
        result_price_mean = result['price_mean'].values 
        result_price_median = result['price_median'].values
        
        result['diff_mean'] = result_price - result_price_mean
        result['diff_median'] = result_price - result_price_median
        result['diff_mean_rel'] = (result_price - result_price_mean)/result_price
        result['diff_median_rel'] = (result_price - result_price_median)/result_price
    
    with Timer('join', profile) as t:
        # fetch the meta data
        result = result.join(meta, on='item_id')
    with Timer('create_mean_meta1', profile) as t:
        result['star_mean'] = np.mean(result['star'].values)
    with Timer('create_mean_meta2', profile) as t:
        result['gr_mean'] = np.mean(result['good_rating'].values)
    with Timer('create_mean_meta3', profile) as t:
        result['sr_mean'] = np.mean(result['satisfactory_rating'].values)
    with Timer('create_mean_meta4', profile) as t:
        result['er_mean'] = np.mean(result['excellent_rating'].values)
    with Timer('create_mean_meta5', profile) as t:
        result.reset_index(inplace=True)
        
    with Timer('create target', profile) as t:
        # get target
        ref = int(last_row['reference'])
        result['target'] = (result['item_id'].values == ref).astype(int)
    return result

n_runs = 5000
n = 0
for k,v in session_grp:
    a = compute_clicks(v)
    n += 1
    if n == n_runs:
        break

# 2) dwell time for each clickout
# 3) the rating and star and nproperty -> mean and median for 
#     the other impressions in list (or the quantile of these and prices)
# 4) location and relative location in the impressions list

CPU times: user 2.82 s, sys: 19.9 ms, total: 2.84 s
Wall time: 2.84 s


In [53]:
get_profile(profile)

,task,sec,per
0,select_rows,0.13890,5.27
1,exclude_nans,0.24707,9.37
2,get_imp_list,0.02312,0.88
3,get_price,0.01837,0.70
4,create_df,0.20587,7.81
5,rel_price_rank,0.14684,5.57
6,compute_mean_median,0.33842,12.84
7,compute_diff,0.03954,1.50
8,join,0.59667,22.63
9,create_mean_meta1,0.14876,5.64


In [41]:
get_profile(profile)

,task,sec,per
0,select_rows,0.13722,4.43
1,exclude_nans,0.24328,7.85
2,get_imp_list,0.02204,0.71
3,get_price,0.01800,0.58
4,create_df,0.20296,6.55
5,rel_price_rank,0.14513,4.68
6,compute_mean_median,0.33419,10.79
7,compute_diff,0.51503,16.63
8,join,0.60704,19.60
9,create_mean_meta1,0.14558,4.70


In [42]:
df = get_profile(profile)

In [43]:
%timeit df['per'].mean()

28.8 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [44]:
%timeit np.mean(df['per'].values)

6.46 µs ± 15.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [45]:
%timeit df['per'].values

1.7 µs ± 20.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [46]:
pprint(profile)

Pretty printing has been turned OFF


In [47]:
%%time
dd = session_grp.apply(compute_clicks)

CPU times: user 3.6 s, sys: 7.97 ms, total: 3.61 s
Wall time: 3.6 s


In [48]:
%%time
# join on session features
dd = dd.reset_index(level=0).set_index('session_id')

CPU times: user 0 ns, sys: 3.36 ms, total: 3.36 ms
Wall time: 3.16 ms


In [49]:
%%time
final = dd.join(session_fts)

CPU times: user 17 ms, sys: 3.95 ms, total: 20.9 ms
Wall time: 20.2 ms


### Questions:

1) timestamp to utc time, timezones? (this matters to e.g. hours of the timestamp)

2) does platform change within a session?

to do:

1) check the time range of train vs test